# Using itk-elastix to register a single section to a BrainGlobe Atlas

[BrainGlobe](https://brainglobe.info/) 2D image registration has not yet been released, so we're going to use the [`itk-elastix`](https://github.com/InsightSoftwareConsortium/ITKElastix) library directly.

## Load the data
We're using [pooch](https://www.fatiando.org/pooch/) to automate downloading the data, and check the hash.

**Run the following cell to download the data in advance**

In [1]:
# Download and load a 2D coronal image
import pooch
import tifffile
from pathlib import Path

# Use pooch to fetch data if it hasn't already been downloaded
image_url = "https://gin.g-node.org/neuroinformatics/image-analysis-courses/raw/master/misc/coronal_section.tif"
image_path = pooch.retrieve(image_url, path=Path.cwd().parent, fname="coronal_section.tif", known_hash="653539933fe1cafcdaa651378f7ebc01beced68298fceac4a8ec1da13f138c43", progressbar=True)
fixed_image = tifffile.imread(image_path)

## View image in napari

In [2]:
import napari
viewer = napari.Viewer()
viewer.add_image(fixed_image, contrast_limits=[0,10000])

<Image layer 'fixed_image' at 0x7f090428b3a0>

## Instantiate a BrainGlobe Atlas API atlas class and choose an appropriate plane

In [3]:
# The first time you use a BrainGlobe atlas, it will need to be downloaded.
# This is done automatically, but it may take some time, depending on the atlas

from bg_atlasapi import BrainGlobeAtlas
atlas_plane = 230 
atlas = BrainGlobeAtlas("allen_mouse_25um")
moving_image = atlas.reference[atlas_plane]
viewer.add_image(moving_image)

<Image layer 'moving_image' at 0x7f08971a39a0>

## Resample so the images are the same scale


In [4]:
from scipy.ndimage import zoom
image_pixel_size = 5 # um
atlas_pixel_size = atlas.resolution[0]
zoom_ratio = image_pixel_size / atlas_pixel_size
downsampled_fixed_image = zoom(fixed_image, zoom_ratio)
viewer.add_image(downsampled_fixed_image, contrast_limits=[0,10000])

<Image layer 'downsampled_fixed_image' at 0x7f088fb4d120>

# Define a registration function 
See `itk-elastix` examples for more


In [5]:
import itk
import numpy as np

def run_registration(
    fixed_image,
    moving_image,
    rigid=True,
    affine=True,
    bspline=True,
    affine_iterations="2048",
    log=False,
):
    # convert to ITK, view only
    fixed_image = itk.GetImageViewFromArray(fixed_image).astype(itk.F)
    moving_image = itk.GetImageViewFromArray(moving_image).astype(itk.F)

    # This syntax needed for 3D images. Leaving here for compatibility.
    elastix_object = itk.ElastixRegistrationMethod.New(
        fixed_image, moving_image
    )

    parameter_object = setup_parameter_object(
        rigid=rigid,
        affine=affine,
        bspline=bspline,
        affine_iterations=affine_iterations,
    )
    elastix_object.SetParameterObject(parameter_object)
    elastix_object.SetLogToConsole(log)

    # update filter object
    elastix_object.UpdateLargestPossibleRegion()

    # get results
    result_image = elastix_object.GetOutput()
    result_transform_parameters = elastix_object.GetTransformParameterObject()
    return np.asarray(result_image), result_transform_parameters


def setup_parameter_object(
    rigid=True,
    affine=True,
    bspline=True,
    affine_iterations="2048",
):
    parameter_object = itk.ParameterObject.New()

    if rigid:
        parameter_map_rigid = parameter_object.GetDefaultParameterMap("rigid")
        parameter_object.AddParameterMap(parameter_map_rigid)

    if affine:
        parameter_map_affine = parameter_object.GetDefaultParameterMap(
            "affine"
        )
        parameter_map_affine["MaximumNumberOfIterations"] = [affine_iterations]
        parameter_object.AddParameterMap(parameter_map_affine)

    if bspline:
        parameter_map_bspline = parameter_object.GetDefaultParameterMap(
            "bspline"
        )
        parameter_object.AddParameterMap(parameter_map_bspline)

    return parameter_object

## Register the atlas image to the sample

In [6]:
registered_image, result_transform_parameters = run_registration(downsampled_fixed_image, moving_image)

## Visualise the result

In [7]:
viewer.add_image(registered_image)

<Image layer 'registered_image' at 0x7f079ae78430>

## Define a function to transform any other images from atlas to sample space

In [8]:
def transform_additional_image(moving_image, result_transform_parameters, mask=False):
    result_transform_parameters.SetParameter('FinalBSplineInterpolationOrder','0')
    
    # convert to ITK, view only
    moving_image = itk.GetImageViewFromArray(moving_image).astype(itk.F)
    
    # Load Transformix Object
    transformix_object = itk.TransformixFilter.New(moving_image)
    transformix_object.SetTransformParameterObject(result_transform_parameters)

    
    # Update object (required)
    transformix_object.UpdateLargestPossibleRegion()
    
    # Results of Transformation
    result_image = transformix_object.GetOutput()

    if mask:
        return np.asarray(result_image, dtype=int)
    else:
        return np.asarray(result_image)

## Apply transformation to the annotations image


In [9]:
warped_annotations = transform_additional_image(atlas.annotation[atlas_plane], result_transform_parameters, mask=True)
viewer.add_labels(warped_annotations, opacity=0.3)

<Labels layer 'warped_annotations' at 0x7f087e477250>

## Upsample the registered annotations to the raw data

In [10]:
zoom_ratio = atlas_pixel_size/ image_pixel_size
# set order=0 for no interpolation,
upsampled_warped_annotations = zoom(warped_annotations, zoom_ratio, order=0)
viewer.add_labels(upsampled_warped_annotations, opacity=0.3)

<Labels layer 'upsampled_warped_annotations' at 0x7f079ae782e0>

## Use the atlas API to analyse the original image

In [11]:
# Pick a value from atlas to inspect
from pprint import pprint
pprint(atlas.structures[945])

{'acronym': 'SSp-ul6a',
 'id': 945,
 'mesh': None,
 'mesh_filename': PosixPath('/home/adam/.brainglobe/allen_mouse_25um_v1.2/meshes/945.obj'),
 'name': 'Primary somatosensory area, upper limb, layer 6a',
 'rgb_triplet': [24, 128, 100],
 'structure_id_path': [997, 8, 567, 688, 695, 315, 453, 322, 369, 945]}
